Source: [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# Influenza Tracking Dashboard

### Data Source and Graph Description

The data used in this dashboard is retrieved from the UK Health Security Agency's Respiratory Viruses Dashboard:  
[UKHSA Dashboard - Influenza](https://ukhsa-dashboard.data.gov.uk/respiratory-viruses/influenza).

#### Graph Content:
- The graph visualises trends in influenza cases over time, based on publicly available UKHSA data accessed through API.
- It highlights the weekly number of testing and hospitalisation cases (including ICU/HDU hospital admissions and general hospital admissions)
- The purpose of this visualization is to help identify patterns and peaks in influenza activity across different age groups in the UK.

**Disclaimer:** The data is subject to change and updates as reported by the UKHSA. Users should consult the source for the most accurate and recent information.


In [1]:
from IPython.display import clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json
from datetime import datetime

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable
with open("tests.json", "rt") as INFILE:
    tests=json.load(INFILE)
with open("icu_admissions.json", "rt") as INFILE:
    icu_admissions=json.load(INFILE)
with open("hospital_admissions.json", "rt") as INFILE:
    hospital_admissions=json.load(INFILE)

In [4]:
# Create API wrapper object to use later for downloading data from UKHSA
class APIwrapper:
    _access_point="https://api.ukhsa-dashboard.data.gov.uk"
    _last_access=0.0 # time of last api access
    
    def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
        url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                  f"{geography_type}/geographies/{geography}/metrics/{metric}")
        self._start_url=APIwrapper._access_point+url_path
        self._filters=None
        self._page_size=-1
        self.count=None

    def get_page(self, filters={}, page_size=5):
        if page_size>365:
            raise ValueError("Max supported page size is 365")
        if filters!=self._filters or page_size!=self._page_size:
            self._filters=filters
            self._page_size=page_size
            self._next_url=self._start_url
        if self._next_url==None: 
            return [] 
        curr_time=time.time() 
        deltat=curr_time - APIwrapper._last_access
        if deltat<0.33: # max 3 requests/second
            time.sleep(0.33-deltat)
        APIwrapper._last_access=curr_time
        parameters={x: y for x, y in filters.items() if y!=None}
        parameters['page_size']=page_size
        response = requests.get(self._next_url, params=parameters).json()
        self._next_url=response['next']
        self.count=response['count']
        return response['results'] 

    def get_all_pages(self, filters={}, page_size=365):
        data=[]
        while True:
            next_page=self.get_page(filters, page_size)
            if next_page==[]:
                break
            data.extend(next_page)
        return data

In [5]:
# Function to get api data 
def access_api(metric):
    """ Accesses the UKHSA API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    structure={"theme": "infectious_disease", 
           "sub_theme": "respiratory",
           "topic": "Influenza",
           "geography_type": "Nation", 
           "geography": "England"}
    structure["metric"] = metric
    api = APIwrapper(**structure)
    data = api.get_all_pages()
    return data # return data read from the API

In [6]:
# Function to wrangle raw data from the API
def wrangle_data(rawdata): 
    data = {}
    for entry in rawdata:
        date = entry['date']
        value = entry['metric_value']
        age = entry['age']
        if age == '0-4':
            age = '00-04' # this is to make age groups format consistent, group 0-4 is same as 00-04
        if age == '5-14':
            age = '05-14' # this is to make age groups format consistent, group 5-14 is same as 05-14
        if date not in data:
            data[date] = {}
        data[date][age] = value
    # Convert data to pandas dataframe
    df = pd.DataFrame.from_dict(data, orient='index')
    # Convert date from string to date format using datetime module
    df.index = df.index.map(lambda x: datetime.strptime(x,"%Y-%m-%d"))
    # Sort index to display df in date order
    df = df.sort_index()
    # Fill all na values with 0
    df.fillna(value=0, inplace=True)
    return df

In [7]:
# Function to plot data, with some parameters for reusability for plotting different datasets
# Parameters selected_year and selected_age are widgets' values
def plot_data(df, selected_year, selected_age, ylabel1, ylabel2):
    age = [age for age in selected_age] # list of all age groups values selected by user when interacting with age widget
    yeardf = df[df.index.year.isin(selected_year)] # filtered df based on year selected by user when interacting with year widget
    fig, ax = plt.subplots(1,2, figsize=(12,6)) 
    # Graph 1: metric values over time across all age
    yeardf['all'].plot(ax=ax[0], color='#235080')
    ax[0].set_ylabel(ylabel1)
    # Graph 2: metric value over time across different age groups
    yeardf[age].plot(ax=ax[1], cmap=plt.cm.Blues)
    ax[1].set_ylabel(ylabel2)
    ax[1].legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    plt.show()

In [8]:
# Function to refresh graph
def refresh_graph(year_widget, age_widget):
    currentyear = year_widget.value
    currentage = age_widget.value
    # Toggle year
    if currentyear == (year_widget.options[-1],):
        otheryear = (year_widget.options[0],)
    else:
        otheryear = (year_widget.options[-1],)
    year_widget.value=otheryear # forces the redraw
    year_widget.value=currentyear # now we can change it back
    # Toggle age
    if currentage == (age_widget.options[0],):
        otherage = (age_widget.options[-1],)
    else:
        otherage = (age_widget.options[0],)
    age_widget.value = otherage # forces the redraw
    age_widget.value = currentage # now we can change it back

In [9]:
# Function to execute when refresh data button is clicked
def api_button_callback(button, metric, output, button_widget, year_widget, age_widget):
    # Get fresh data from the API. If you have time, include some error handling around this call.
    try: 
        apidata = access_api(metric)
        df = wrangle_data(apidata)
        with output:
            refresh_graph(year_widget, age_widget) # the graph won't refresh until the user interacts with the widget.
            button_widget.icon="check"
    # Handle connection-related issues (i.e., network down or server unreachable)
    except ConnectionError:
        with output:
            print("Error: Unable to connect to the data. Please check your internet connection or try again later.")
            button_widget.icon='unlink'
    # Handle data wrangling errors (i.e., unexpected API response structure)
    except ValueError as e:
        with output:
            print(f"Error: Data processing failed due to an unexpected value: {e}")
            button_widget.icon="times"
    # Catch any other unforeseen errors
    except Exception as e:
        with output:
            print(f"An unexpected error occurred: {e}")
            button_widget.icon="times"
            
    finally:
        button_widget.button_style=''
        button_widget.disabled=True

## <font color="#4384f3">Testing: Weekly positivity of people receiving a PCR test</font>

**<font color="#555555">The two figures below show the percentage of people who received a PCR and had at least one positive PCR test result for influenza in the same 7 days. Data is shown by specimen date (the date the sample was collected)</font>**

_Source: https://ukhsa-dashboard.data.gov.uk/respiratory-viruses/influenza_

- Select the year or age groups for which you want to see the data. You can select multiple years or age groups by Shift+Click.
- To get the most recent data from Public Health England, please click Refresh data

In [10]:
tests_df = wrangle_data(tests)

In [11]:
# Create button widgets
button_wdg1 = widgets.Button(
    description = 'Refresh data',
    button_style = 'primary',
    disabled = False,
    tooltip = "Click to get current Public Health England data",
    icon='refresh'
)
# Create year widgets
year_wdg1 = widgets.SelectMultiple(
    options = tests_df.index.year.unique(), # options available
    value = [tests_df.index.year[-1]], # initial value: most recent year
    description = 'Select year',
    disabled = False
)
# Create age widgets
age_wdg1 = widgets.SelectMultiple(
    options = tests_df.columns,
    value = [col for col in tests_df.columns if col != 'all'],
    description = 'Select age',
    disabled = False
)

In [12]:
# Capture output in widget output    
tests_output = widgets.interactive_output(plot_data, {
    'selected_year': year_wdg1, 
    'selected_age': age_wdg1, 
    'df': widgets.fixed(tests_df), 
    'ylabel1': widgets.fixed('Weekly PCR Test Positivity (%)'), 
    'ylabel2': widgets.fixed('Weekly PCR Test Positivity by Age Groups (%)')})

display(button_wdg1, year_wdg1, age_wdg1, tests_output)

Button(button_style='primary', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click…

SelectMultiple(description='Select year', index=(7,), options=(2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024)…

SelectMultiple(description='Select age', index=(0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), options=('00-04', …

Output()

In [13]:
#  Register button callback function with the button
button_wdg1.on_click(lambda b: api_button_callback(b, 
                                                   metric='influenza_testing_positivityByWeek', 
                                                   output=tests_output, 
                                                   button_widget=button_wdg1, 
                                                   year_widget=year_wdg1, 
                                                   age_widget=age_wdg1)) 

## <font color="#4384f3">Healthcare: ICU HDU admission rates per 100,000 population</font>

**<font color="#555555">The two figures below show weekly overall influenza hospital ICU HDU rates per 100,000 trust catchment population</font>**

_Source: https://ukhsa-dashboard.data.gov.uk/respiratory-viruses/influenza_

- Select the year or age groups for which you want to see the data. You can select multiple years or age groups by Shift+Click.
- To get the most recent data from Public Health England, please click Refresh data

In [14]:
icu_df = wrangle_data(icu_admissions)

In [15]:
# Create button widgets
button_wdg2 = widgets.Button(
    description = 'Refresh data', 
    button_style = 'primary',
    disabled = False,
    tooltip = "Click to get current Public Health England data",
    icon='refresh'
)
# Create year widgets
year_wdg2 = widgets.SelectMultiple(
    options = icu_df.index.year.unique(), # options available
    value = [icu_df.index.year[-1]], # initial value: most recent year
    description = 'Select year',
    disabled = False,
)
# Create age widgets
age_wdg2 = widgets.SelectMultiple(
    options = icu_df.columns,
    value = [col for col in icu_df.columns if col != 'all'],
    description = 'Select age',
    disabled = False
)

In [16]:
# Capture output in widget output    
icu_output = widgets.interactive_output(plot_data, {
    'selected_year': year_wdg2, 
    'selected_age': age_wdg2, 
    'df': widgets.fixed(icu_df), 
    'ylabel1': widgets.fixed('Weekly Rates per 100,000 Population'), 
    'ylabel2': widgets.fixed('Weekly Rates per 100,000 Population by Age Groups')})

display(button_wdg2, year_wdg2, age_wdg2, icu_output)

Button(button_style='primary', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click…

SelectMultiple(description='Select year', index=(9,), options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,…

SelectMultiple(description='Select age', index=(1, 2, 3, 4, 5, 6, 7, 8, 9), options=('all', '65-74', '45-54', …

Output()

In [17]:
# Register button callback function with the button
button_wdg2.on_click(lambda b: api_button_callback(b, 
                                                   metric='influenza_healthcare_ICUHDUadmissionRateByWeek', 
                                                   output=icu_output, 
                                                   button_widget=button_wdg2, 
                                                   year_widget=year_wdg2, 
                                                   age_widget=age_wdg2)) 

## <font color="#4384f3">Healthcare: Hospital admission rates per 100,000 population</font>

**<font color="#555555">The two figures below show Weekly overall influenza hospital admission rates per 100,000 trust catchment population</font>**

_Source: https://ukhsa-dashboard.data.gov.uk/respiratory-viruses/influenza_

- Select the year or age groups for which you want to see the data. You can select multiple years or age groups by Shift+Click.
- To get the most recent data from Public Health England, please click Refresh data

In [18]:
hospital_df = wrangle_data(hospital_admissions)

In [19]:
# Create button widgets
button_wdg3 = widgets.Button(
    description = 'Refresh data',
    button_style = 'primary',
    disabled = False,
    tooltip = "Click to get current Public Health England data",
    icon='refresh'
)
# Create year widgets
year_wdg3 = widgets.SelectMultiple(
    options = hospital_df.index.year.unique(), # options available
    value = [hospital_df.index.year[-1]], # initial value: most recent year
    description = 'Select year',
    disabled = False,
)
# Create age widgets
age_wdg3 = widgets.SelectMultiple(
    options = hospital_df.columns,
    value = [col for col in hospital_df.columns if col != 'all'],
    description = 'Select age',
    disabled = False
)

In [20]:
# Capture output in widget output    
hospital_output = widgets.interactive_output(plot_data, {
    'selected_year': year_wdg3, 
    'selected_age': age_wdg3, 
    'df': widgets.fixed(hospital_df), 
    'ylabel1': widgets.fixed('Weekly Rates per 100,000 Population'), 
    'ylabel2': widgets.fixed('Weekly Rates per 100,000 Population by Age Groups')})

display(button_wdg3, year_wdg3, age_wdg3, hospital_output)

Button(button_style='primary', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click…

SelectMultiple(description='Select year', index=(9,), options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,…

SelectMultiple(description='Select age', index=(1, 2, 3, 4, 5, 6, 7, 8, 9), options=('all', '65-74', '45-54', …

Output()

In [21]:
#  Register button callback function with the button
button_wdg3.on_click(lambda b: api_button_callback(b, 
                                                   metric="influenza_healthcare_hospitalAdmissionRateByWeek", 
                                                   output=icu_output, 
                                                   button_widget=button_wdg3, 
                                                   year_widget=year_wdg3, 
                                                   age_widget=age_wdg3)) 

**Author and License**: Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)